In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices
from linearmodels.panel import PanelOLS

In [4]:
ICT = pd.read_csv('ICT_data.csv')

ICT=ICT.rename(columns={'2010 [YR2010]':2010,'2011 [YR2011]':2011,'2012 [YR2012]':2012,\
                    '2013 [YR2013]':2013,'2014 [YR2014]':2014,'2015 [YR2015]':2015,\
                    '2016 [YR2016]':2016,'2017 [YR2017]':2017,'2018 [YR2018]':2018,\
                    '2019 [YR2019]':2019})
    
for i in ICT.columns[4:]:
    ICT[i]=pd.to_numeric(ICT[i],errors='coerce')
    
ICT.drop([1320,1321,1322,1323,1324],inplace=True)

ICTf = ICT.fillna(method='ffill',axis=1)

ICT=ICT.drop(['Country Code','Series Code'],axis=1)
ICT=ICT.set_index(['Country Name','Series Name'])

ICTf=ICTf.drop(['Country Code','Series Code'],axis=1)
ICTf.set_index(['Series Name'],inplace=True)

FileNotFoundError: [Errno 2] File ICT_data.csv does not exist: 'ICT_data.csv'

In [19]:
ICT.columns.rename('year',inplace=True)

In [20]:
ICT=ICT.stack().swaplevel(1,2).unstack()

In [21]:
WGI = pd.read_csv('/Users/chenruoyu/Desktop/DB project data/WGI.csv')

WGI=WGI.rename(columns={'2009 [YR2009]':2009,'2010 [YR2010]':2010,'2011 [YR2011]':2011,'2012 [YR2012]':2012,\
                    '2013 [YR2013]':2013,'2014 [YR2014]':2014,'2015 [YR2015]':2015,\
                    '2016 [YR2016]':2016,'2017 [YR2017]':2017,'2018 [YR2018]':2018})
for i in WGI.columns[4:]:
    WGI[i]=pd.to_numeric(WGI[i],errors='coerce')
    
WGI.drop([1283,1284,1285,1286,1287,1288],inplace=True)
WGI.drop(['2004 [YR2004]','2005 [YR2005]','2006 [YR2006]','2007 [YR2007]','2008 [YR2008]'],axis=1,inplace=True)

WGI=WGI.drop(['Country Code','Series Code'],axis=1)
WGI=WGI.set_index(['Country Name','Series Name'])
WGI.columns.rename('year',inplace=True)

In [22]:
WGI=WGI.stack().swaplevel(1,2).unstack()

In [51]:
ICT.index.rename(['country_name','time'],inplace=True)
WGI_ICT=WGI.merge(ICT, how='left', left_index=True, right_on=['country_name','time'])
WGI_ICT = WGI_ICT.swaplevel(0,1)

In [52]:
WGI_ICT.corr()

Series Name,Control of Corruption: Percentile Rank,Government Effectiveness: Percentile Rank,Political Stability and Absence of Violence/Terrorism: Percentile Rank,Regulatory Quality: Percentile Rank,Rule of Law: Percentile Rank,Voice and Accountability: Percentile Rank,Fixed broadband subscriptions (per 100 people),"ICT service exports (BoP, current US$)",Individuals using the Internet (% of population),Mobile cellular subscriptions (per 100 people),Secure Internet servers (per 1 million people)
Series Name,,,,,,,,,,,
Control of Corruption: Percentile Rank,1.000000,0.902161,0.784812,0.841360,0.939679,0.791499,0.731850,0.275431,0.728024,0.466062,0.253087
Government Effectiveness: Percentile Rank,0.902161,1.000000,0.690982,0.923381,0.923455,0.731254,0.805260,0.318693,0.820452,0.582236,0.257242
Political Stability and Absence of Violence/Terrorism: Percentile Rank,0.784812,0.690982,1.000000,0.620285,0.783504,0.726904,0.598369,0.092942,0.575322,0.367006,0.210505
Regulatory Quality: Percentile Rank,0.841360,0.923381,0.620285,1.000000,0.888411,0.733252,0.790636,0.280292,0.796347,0.591685,0.242911
Rule of Law: Percentile Rank,0.939679,0.923455,0.783504,0.888411,1.000000,0.818734,0.764500,0.308687,0.765609,0.495342,0.244413
Voice and Accountability: Percentile Rank,0.791499,0.731254,0.726904,0.733252,0.818734,1.000000,0.677455,0.276477,0.587014,0.319121,0.224057
Fixed broadband subscriptions (per 100 people),0.731850,0.805260,0.598369,0.790636,0.764500,0.677455,1.000000,0.308908,0.855945,0.456354,0.326696
"ICT service exports (BoP, current US$)",0.275431,0.318693,0.092942,0.280292,0.308687,0.276477,0.308908,1.000000,0.243150,0.045001,0.179127
Individuals using the Internet (% of population),0.728024,0.820452,0.575322,0.796347,0.765609,0.587014,0.855945,0.243150,1.000000,0.610742,0.317804


In [53]:
WGI_ICT.drop('ICT service exports (BoP, current US$)',axis=1,inplace=True)

In [54]:
df=WGI_ICT.dropna()

In [55]:
df=df.rename(columns={'Control of Corruption: Percentile Rank':'corruption','Government Effectiveness: Percentile Rank':'effectiveness',
                  'Political Stability and Absence of Violence/Terrorism: Percentile Rank':'stability','Regulatory Quality: Percentile Rank':'regulation',
                  'Rule of Law: Percentile Rank':'law','Voice and Accountability: Percentile Rank':'voice','Fixed broadband subscriptions (per 100 people)':'broadband',
                  'ICT service exports (BoP, current US$)':'export','Individuals using the Internet (% of population)':'user',
                  'Mobile cellular subscriptions (per 100 people)':'mobile','Secure Internet servers (per 1 million people)':'server'})

In [56]:
y, X = dmatrices('corruption ~  broadband + mobile + user +server', data=df, return_type='dataframe')

In [57]:
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             corruption   R-squared:                       0.574
Model:                            OLS   Adj. R-squared:                  0.573
Method:                 Least Squares   F-statistic:                     521.1
Date:                Tue, 21 Apr 2020   Prob (F-statistic):          1.58e-284
Time:                        17:37:07   Log-Likelihood:                -6736.9
No. Observations:                1549   AIC:                         1.348e+04
Df Residuals:                    1544   BIC:                         1.351e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     21.6353      1.335     16.202      0.0

In [61]:
df=df.swaplevel(0,1)

In [62]:
df

Series Name        corruption  effectiveness  stability  regulation  \
country_name time                                                     
Afghanistan  2010    0.952381       4.784689   1.421801    5.263158   
             2012    2.369668       6.635071   1.895735   10.900470   
             2013    1.895735       7.582938   1.421801   10.900470   
             2014    5.288462       8.173077   2.857143   11.057690   
             2015    6.250000       8.173077   0.952381   14.423080   
...                       ...            ...        ...         ...   
Zambia       2014   45.192310      33.173080  52.380950   32.692310   
             2015   45.673080      32.692310  53.333330   36.538460   
             2016   41.346150      27.403850  51.428570   32.692310   
             2017   34.615380      28.846150  52.380950   33.653850   
             2018   27.884610      33.173080  53.333330   35.096150   

Series Name              law      voice  broadband       user     mobile  \
country_name time                                                          
Afghanistan  2010   0.473934   9.952606   0.005140   4.000000  35.003127   
             2012   1.408451  14.084510   0.004814   5.454545  49.227977   
             2013   1.408451  14.553990   0.004648   5.900000  52.083576   
             2014   4.326923  16.256160   0.004495   7.000000  55.159515   
             2015   3.365385  18.719210   0.020535   8.260000  57.271068   
...                      ...        ...        ...        ...        ...   
Zambia       2014  47.115380  44.334980   0.133255  19.000000  65.681859   
             2015  47.115380  45.812810   0.147298  21.000000  72.784572   
             2016  43.269230  35.960590   0.194238  25.506579  73.438230   
             2017  41.346150  35.960590   0.213082  27.852579  79.736910   
             2018  40.865380  35.960590   0.249918  14.299997  89.157044   

Series Name           server  
country_name time             
Afghanistan  2010   0.479690  
             2012   1.219458  
             2013   1.425491  
             2014   1.708080  
             2015   2.150312  
...                      ...  
Zambia       2014   4.090975  
             2015   5.667734  
             2016   8.861181  
             2017  42.364615  
             2018  41.206047  

[1549 rows x 10 columns]

In [63]:
y, X = dmatrices('corruption ~  broadband + mobile + user +server', data=df, return_type='dataframe')
mod = PanelOLS(y, X, entity_effects=True, time_effects=True)
res = mod.fit(cov_type='unadjusted')
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:             corruption   R-squared:                        0.0219
Estimator:                   PanelOLS   R-squared (Between):              0.1645
No. Observations:                1549   R-squared (Within):              -0.0223
Date:                Tue, Apr 21 2020   R-squared (Overall):              0.1609
Time:                        17:40:44   Log-likelihood                   -4371.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      7.5090
Entities:                         213   P-value                           0.0000
Avg Obs:                       7.2723   Distribution:                  F(4,1344)
Min Obs:                       0.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             7.5090
                            

In [64]:
y, X = dmatrices('effectiveness ~  broadband + mobile + user +server', data=df, return_type='dataframe')
mod = PanelOLS(y, X, entity_effects=True, time_effects=True)
res = mod.fit(cov_type='unadjusted')
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:          effectiveness   R-squared:                        0.0254
Estimator:                   PanelOLS   R-squared (Between):              0.3116
No. Observations:                1549   R-squared (Within):              -0.0589
Date:                Tue, Apr 21 2020   R-squared (Overall):              0.3080
Time:                        17:42:12   Log-likelihood                   -4453.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      8.7589
Entities:                         213   P-value                           0.0000
Avg Obs:                       7.2723   Distribution:                  F(4,1344)
Min Obs:                       0.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             8.7589
                            

In [65]:
y, X = dmatrices('stability ~  broadband + mobile + user +server', data=df, return_type='dataframe')
mod = PanelOLS(y, X, entity_effects=True, time_effects=True)
res = mod.fit(cov_type='unadjusted')
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:              stability   R-squared:                        0.0101
Estimator:                   PanelOLS   R-squared (Between):              0.2028
No. Observations:                1549   R-squared (Within):              -0.0063
Date:                Tue, Apr 21 2020   R-squared (Overall):              0.1959
Time:                        17:42:31   Log-likelihood                   -4984.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3.4332
Entities:                         213   P-value                           0.0084
Avg Obs:                       7.2723   Distribution:                  F(4,1344)
Min Obs:                       0.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             3.4332
                            

In [66]:
y, X = dmatrices('regulation ~  broadband + mobile + user +server', data=df, return_type='dataframe')
mod = PanelOLS(y, X, entity_effects=True, time_effects=True)
res = mod.fit(cov_type='unadjusted')
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:             regulation   R-squared:                        0.0342
Estimator:                   PanelOLS   R-squared (Between):              0.2314
No. Observations:                1549   R-squared (Within):              -0.0363
Date:                Tue, Apr 21 2020   R-squared (Overall):              0.2283
Time:                        17:42:50   Log-likelihood                   -4098.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      11.885
Entities:                         213   P-value                           0.0000
Avg Obs:                       7.2723   Distribution:                  F(4,1344)
Min Obs:                       0.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             11.885
                            

In [67]:
y, X = dmatrices('law ~  broadband + mobile + user +server', data=df, return_type='dataframe')
mod = PanelOLS(y, X, entity_effects=True, time_effects=True)
res = mod.fit(cov_type='unadjusted')
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                    law   R-squared:                        0.0314
Estimator:                   PanelOLS   R-squared (Between):              0.2052
No. Observations:                1549   R-squared (Within):               0.0069
Date:                Tue, Apr 21 2020   R-squared (Overall):              0.2026
Time:                        17:43:13   Log-likelihood                   -4148.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      10.878
Entities:                         213   P-value                           0.0000
Avg Obs:                       7.2723   Distribution:                  F(4,1344)
Min Obs:                       0.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             10.878
                            